In [ ]:
#@title Download model
%%capture

import os

!pip install git+https://github.com/YardenReich/DL-Project.git

import gdown

url = "https://drive.google.com/uc?export=download&id=1uNS2G3908KpiJ7sPfLYev36NTGJHuptM"
output = "model_first.pt"
gdown.download(url, output, quiet=True)

url = "https://drive.google.com/uc?export=download&id=1SZYE7CDWMJehD0MN0c8inp9e-IZKRHq1"
output = "model_second.pt"
gdown.download(url, output, quiet=True)

url = "https://drive.google.com/uc?export=download&id=1FaDEveOI1Gotf1YTP_aP-4yqH5PIdnCp"
output = "data.csv"
gdown.download(url, output, quiet=True)

url = "https://drive.google.com/file/d/1rZ2PMevI54zA7VaKV37Euh66SE-Hh060"
output = "image1.jpg"
gdown.download(url, output, quiet=True)

url = "https://drive.google.com/file/d/1wV5Y3z96vNlRlJXhy0TuO8AYsMdyF7i6"
output = "image2.jpg"
gdown.download(url, output, quiet=True)

os.makedirs("images", exist_ok=True)
url = "https://drive.google.com/drive/folders/1aQ0vBOZ_aJr_LhEiJgakCGMnu2nDBE08"
output = "./images/burn_images"
gdown.download_folder(url=url, output=output, quiet=True)





In [ ]:
#@title Imports
import interpolation
import argparse
import torch
import ipywidgets as widgets

In [ ]:
#@title To upload your images
uploader = widgets.FileUpload(
    description='Pick files',
    accept='.jpg,.png, .jpeg',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)

button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)

display(uploader)
display(button)

#@title To save the upload
def handle_upload(filename, uploader):
    filedata = uploader.value[filename]['content']

    with open(filename, 'wb') as f:
        f.write(filedata)

button.on_click(lambda _: [handle_upload(filename, uploader) for filename in uploader.value.keys()])


# for image_name in uploader.value.keys():
#   handle_upload(image_name, uploader)

In [ ]:
#@title Parameters { run: "auto", display-mode: "form" }
#@markdown ### options:

#@markdown Video name:
video_name = "video" #@param {type:"string"}

#@markdown First image path:
csv_path = "data.csv" #@param {type:"string"}

#@markdown First image path:
image1_path = "image1.jpg" #@param {type:"string"}

#@markdown Second image path:
image2_path = "image2.jpg" #@param {type:"string"}

#@markdown Image size:
image_size = 480 #@param {type:"integer"}

#@markdown Use the fisrt decoder:
model = "1" #@param [1, 2]

#@markdown Country of the data:
#@markdown *First letter upper case*
country = "Israel" #@param {type:"string"}

#@markdown Use VAE attribute manipulation:
#@markdown *Doesn't use the second image*
attribute_manipulation = False #@param {type: "boolean"}

#@markdown Type of interpolation:
inter_type = 3 #@param [0, 1, 2, 3]

#@markdown Quantize:
quantize = True #@param {type:"boolean"}


#@markdown ### Other Options

#@markdown Frames per second:
#@markdown *min:10 max:60*
fps = 10 #@param {type:"integer"}

#@markdown Add the years:
add_text = False #@param {type: "boolean"}

#@markdown Should the video be saved:
save_video = True #@param {type:"boolean"}

#@markdown Store the images:
save_images = True #@param {type:"boolean"}



model = int(model)
if model == 1:
  model_path = "model_first.pt"
  first_decoder = 1
else:
  model_path = "model_second.pt"
  first_decoder = 0

In [ ]:
#@title Parse args

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

args = argparse.Namespace(
    csv_path = csv_path,
    file_name = video_name,
    image1_path = image1_path,
    image2_path = image2_path,
    model_path = model_path,
    device = device,
    fps = fps,
    num_codebook_vectors = 1024,
    image_size = image_size,
    latent_dim = 256,
    beta = 0.25,
    image_channels = 3,
    batch_size = 1,
    interpolation_type = int(inter_type),
    country = country,
    create_video = save_video,
    save_images = save_images,
    quantize = quantize,
    first_decoder = first_decoder,
    attribute = attribute_manipulation,
    add_text = add_text,
)


In [ ]:
#@title Run Interpolation
interpolation.run_interpolation(args)

In [ ]:
# @title Display the video
import os
from base64 import b64encode
from IPython.display import HTML
mp4 = open(os.path.join("videos", video_name + '.mp4'),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)